In [2]:
import json
import pennylane as qml
import pennylane.numpy as np

def potential_energy_surface(symbols, bond_lengths):
    """Calculates the molecular energy over various bond lengths (AKA the 
    potential energy surface) using the Hartree Fock method.
    
    Args:
        symbols (list(string)): 
            A list of atomic symbols that comprise the diatomic molecule of interest.
        bond_lengths (numpy.tensor): Bond lengths to calculate the energy over.

        
    Returns:
        hf_energies (numpy.tensor): 
            The Hartree Fock energies at every bond length value.
    """


    hf_energies = []

    # Put your code here #
    for bond_length in bond_lengths:
        dev = qml.device("default.qubit", wires=len(symbols))


        @qml.qnode(dev)
        def circuit(bond_length):
            for i, symbol in enumerate(symbols):
                qml.RX(np.pi, wires=i)
                qml.templates.AllSwap(wires=range(len(symbols)))

            for i in range(len(symbols) - 1):
                qml.BasisState(np.array([1, 1], requires_grad=False), wires=[i, i + 1])
                qml.DoubleExcitationUnitary(bond_length, wires=[i, i + 1])

            return qml.expval(qml.Hermitian(np.eye(2 ** len(symbols)), wires=range(len(symbols))))


    energy = circuit(bond_length)
    hf_energies.append(energy)
    

    return np.array(hf_energies)


def ground_energy(hf_energies):
    """Finds the minimum energy of a molecule given its potential energy surface.
    
    Args: 
        hf_energies (numpy.tensor): 

    Returns:
        (float): The minumum energy in units of hartrees.
    """

    ind = np.argmin(hf_energies)
    return hf_energies[ind]

def reaction():
    """Calculates the energy of the reactants, the activation energy, and the energy of 
    the products in that order.

    Returns:
        (numpy.tensor): [E_reactants, E_activation, E_products]
    """
    molecules = {
        "H2": 
            {"symbols": ["H", "H"], "E0": 0, "E_dissociation": 0, "bond lengths": np.arange(0.5, 9.3, 0.3)}, 
        "Li2": 
            {"symbols": ["Li", "Li"], "E0": 0, "E_dissociation": 0, "bond lengths": np.arange(3.5, 8.3, 0.3)}, 
        "LiH": 
            {"symbols": ["Li", "H"], "E0": 0, "E_dissociation": 0, "bond lengths": np.arange(2.0, 6.6, 0.3)}
    }


    for molecule in molecules.keys():
        # Put your code here #
        symbols = data["symbols"]
        bond_lengths = data["bond lengths"]
        hf_energies = potential_energy_surface(symbols, bond_lengths)

        data["E0"] = hf_energies[0]
        data["E_dissociation"] = hf_energies[-1]

        # populate each molecule's E0 and E_dissociation values

    # Calculate the following and don't forget to balance the chemical reaction!
    E_reactants = molecules["H2"]["E0"] + molecules["Li2"]["E0"]
    E_activation = molecules["H2"]["E_dissociation"] - E_reactants
    E_products = molecules["LiH"]["E_dissociation"] + molecules["Li2"]["E_dissociation"]


    return np.array([E_reactants, E_activation, E_products])


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    output = reaction().tolist()
    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    assert np.allclose(solution_output, expected_output, rtol=1e-3)
